In [6]:
import requests
import pandas as pd
import json
import UpdatingJobTItlesHeaders as U


In [5]:
payload={}
response = requests.request("GET", U.WorkDayurl, headers=U.WorkDayheaders, data=payload)
json_response=response.json()
WorkDayDF= pd.DataFrame.from_dict(json_response['Report_Entry'])

In [2]:
pd.set_option("display.max_rows", None)

In [ ]:

payload={}
response = requests.get(U.Cosentialurl, headers=U.headers_dict, data=payload)
json_response = response.json()

APIdf = pd.DataFrame.from_dict(json_response)


response2 = requests.get(U.Cosentialurl2, headers=U.headers_dict, data=payload)
json_response2 = response2.json()

APIdf2 = pd.DataFrame.from_dict(json_response2)

DataFrame = pd.concat([APIdf,APIdf2])
SmallDataFrame = DataFrame[['PersonnelId', 'OfficeEmail', 'FirstName', 'LastName', 'Status', 'Title']]
ActiveCosentialdf = SmallDataFrame[SmallDataFrame['Status']=='Active']
ActiveCosentialdf['OfficeEmail']= ActiveCosentialdf['OfficeEmail'].str.lower()

In [ ]:
InnerJoinDF = pd.merge(WorkDayDF, ActiveCosentialdf, left_on='Email', right_on='OfficeEmail', how='inner')
InnerJoinDF = InnerJoinDF.rename(columns={'Title': 'Old Job Title', 'JobTitle':'New Job Title'} )
NotMatchingJob = InnerJoinDF[InnerJoinDF['Old Job Title'] !=InnerJoinDF['New Job Title']]
NotMatchingJob = NotMatchingJob[NotMatchingJob['Email'].notnull()]
NotMatchingJob

In [16]:
file_name = 'Non-matching job titles Workday.csv'
NotMatchingJob.to_csv(file_name)

In [ ]:
WorkdayDFtoDict= InnerJoinDF.to_dict('records')
WorkdayNotMatchingJobDict=NotMatchingJob.to_dict('records')
WorkdayNotMatchingJobDict

In [ ]:
i=0
for i in range(len(WorkdayNotMatchingJobDict)):
    payload = json.dumps({"Title":WorkdayNotMatchingJobDict[i]['New Job Title']})
    Testresponse = requests.put(f"{U.Cosentialurl3}{WorkdayNotMatchingJobDict[i]['PersonnelId']}", headers=U.headers_dict, 
    data=payload)
    json_Testresponse = Testresponse.json()
    print(json_Testresponse['FirstName'],json_Testresponse['LastName'],json_Testresponse['PersonnelId'], json_Testresponse['Title'])